In [41]:
from   ast import literal_eval
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from   scipy.stats import permutation_test
import seaborn

## Load derived data

In [57]:
data_dir = os.path.join('..', 'data')
derived_dir = os.path.join(data_dir, 'derived')
df = pd.read_csv(
    os.path.join(derived_dir, 'CONLIT_CharData_AP.csv.gz'), 
    index_col='book_id',
    #converters={
    #    'gpe_places': literal_eval,
    #    'nongpe_places': literal_eval,
    #    'all_places': literal_eval,
    #    'gpe_sequences': literal_eval
    #}
)
df.head()

,char_id,char_count,inf_gender,gpe_places,num_gpe_places,nongpe_places,num_nongpe_places,all_places,num_all_places,gpe_sequences,...,char_rank,num_words,Category,Genre,Tokens,num_gpe_places_norm,num_nongpe_places_norm,num_gpe_places_norm_byCharacter,num_nongpe_places_norm_byCharacter,char_count_norm
book_id,,,,,,,,,,,,,,,,,,,,,
[Heist Society 2] Uncommon Criminals - Ally Carter,138,3238,she/her,"['Paraguay', 'Paraguay', 'London', 'New York C...",15,"['there', 'there', 'there', 'the museum', 'the...",103,"['there', 'there', 'there', 'Paraguay', 'Parag...",118,"['Paraguay', 'London', 'New York City', 'midto...",...,1,320000,FIC,YA,80392,0.000187,0.001281,0.004632,0.031810,0.040278
"2001_2011_Wilson,RobertCharles_TheChronoliths_SF",0,2742,she/her,"['Bangkok', 'Minneapolis / St. Paul', 'Chumpho...",22,"['our household', 'a gas station hawng nam', '...",122,"['our household', 'a gas station hawng nam', '...",143,"['Bangkok', 'Minneapolis / St. Paul', 'Chumpho...",...,1,320000,FIC,SF,103230,0.000213,0.001182,0.008023,0.044493,0.026562
"2001_Martel,Yann_LifeofPi_BS",0,4754,he/him/his,"['Canada', 'India', 'India', 'India', 'India',...",15,"['the north', 'the rich , noisy , functioning ...",142,"['Canada', 'India', 'the north', 'India', 'Ind...",157,"['Canada', 'India', 'Canada', 'India', 'Toront...",...,1,320000,FIC,BS,124445,0.000121,0.001141,0.003155,0.029870,0.038202
"2002_2011_Anderson,MT_Feed_SF",0,2862,she/her,"['Lonely', 'School', 'School', 'Switzerland', ...",5,"['the craters', 'a crummy hotel', 'here', 'her...",68,"['the craters', 'a crummy hotel', 'here', 'her...",72,"['Lonely', 'School', 'Switzerland', 'Io', 'Ame...",...,1,320000,FIC,SF,67075,0.000075,0.001014,0.001747,0.023760,0.042669
"2002_Baker,Jo_Offcomer_CT",120,4723,she/her,"['Belfast', 'Belfast', 'Belfast', 'Belfast', '...",11,"['the bath', 'the bath', 'the bath', 'it', 'th...",162,"['the bath', 'the bath', 'the bath', 'it', 'th...",172,"['Belfast', 'Conroys', 'October', 'Belfast', '...",...,1,320000,FIC,NYT,88042,0.000125,0.001840,0.002329,0.034300,0.053645


In [44]:
df.head()

,char_id,char_count,inf_gender,gpe_places,num_gpe_places,nongpe_places,num_nongpe_places,all_places,num_all_places,gpe_sequences,...,char_rank,num_words,Category,Genre,Tokens,num_gpe_places_norm,num_nongpe_places_norm,num_gpe_places_norm_byCharacter,num_nongpe_places_norm_byCharacter,char_count_norm
book_id,,,,,,,,,,,,,,,,,,,,,
[Heist Society 2] Uncommon Criminals - Ally Carter,138,3238,she/her,"['Paraguay', 'Paraguay', 'London', 'New York C...",15,"['there', 'there', 'there', 'the museum', 'the...",103,"['there', 'there', 'there', 'Paraguay', 'Parag...",118,"['Paraguay', 'London', 'New York City', 'midto...",...,1,320000,FIC,YA,80392,0.000187,0.001281,0.004632,0.031810,0.040278
"2001_2011_Wilson,RobertCharles_TheChronoliths_SF",0,2742,she/her,"['Bangkok', 'Minneapolis / St. Paul', 'Chumpho...",22,"['our household', 'a gas station hawng nam', '...",122,"['our household', 'a gas station hawng nam', '...",143,"['Bangkok', 'Minneapolis / St. Paul', 'Chumpho...",...,1,320000,FIC,SF,103230,0.000213,0.001182,0.008023,0.044493,0.026562
"2001_Martel,Yann_LifeofPi_BS",0,4754,he/him/his,"['Canada', 'India', 'India', 'India', 'India',...",15,"['the north', 'the rich , noisy , functioning ...",142,"['Canada', 'India', 'the north', 'India', 'Ind...",157,"['Canada', 'India', 'Canada', 'India', 'Toront...",...,1,320000,FIC,BS,124445,0.000121,0.001141,0.003155,0.029870,0.038202
"2002_2011_Anderson,MT_Feed_SF",0,2862,she/her,"['Lonely', 'School', 'School', 'Switzerland', ...",5,"['the craters', 'a crummy hotel', 'here', 'her...",68,"['the craters', 'a crummy hotel', 'here', 'her...",72,"['Lonely', 'School', 'Switzerland', 'Io', 'Ame...",...,1,320000,FIC,SF,67075,0.000075,0.001014,0.001747,0.023760,0.042669
"2002_Baker,Jo_Offcomer_CT",120,4723,she/her,"['Belfast', 'Belfast', 'Belfast', 'Belfast', '...",11,"['the bath', 'the bath', 'the bath', 'it', 'th...",162,"['the bath', 'the bath', 'the bath', 'it', 'th...",172,"['Belfast', 'Conroys', 'October', 'Belfast', '...",...,1,320000,FIC,NYT,88042,0.000125,0.001840,0.002329,0.034300,0.053645


In [3]:
df.columns

Index(['char_id', 'char_count', 'inf_gender', 'gpe_places', 'num_gpe_places',
       'nongpe_places', 'num_nongpe_places', 'all_places', 'num_all_places',
       'gpe_sequences', 'dist_miles', 'char_rank', 'num_words', 'Category',
       'Genre', 'Tokens', 'num_gpe_places_norm', 'num_nongpe_places_norm',
       'num_gpe_places_norm_byCharacter', 'num_nongpe_places_norm_byCharacter',
       'char_count_norm'],
      dtype='object')

In [4]:
df.Category.value_counts()

Category
FIC    1934
NON     820
Name: count, dtype: int64

In [5]:
df.Genre.value_counts()

Genre
NYT     419
PW      258
BS      249
MY      234
MEM     229
SF      223
ROM     208
HIST    205
MIX     193
BIO     193
YA      177
MID     166
Name: count, dtype: int64

## Feature differences by facet

In [6]:
facet_by = ['inf_gender', 'Category', 'Genre']
cols_to_test = [
    'char_count',
    'num_gpe_places',
    'num_nongpe_places',
    'num_all_places',
    'dist_miles', 
    'Tokens', 
    'num_gpe_places_norm', 
    'num_nongpe_places_norm',
    'num_gpe_places_norm_byCharacter', 
    'num_nongpe_places_norm_byCharacter',
    'char_count_norm'
]

def significance_label(pvalue, levels=[0.05, 0.01, 0.001]):
    if pvalue < levels[2]:
        return '***'
    elif pvalue < levels[1]:
        return '**'
    elif pvalue < levels[0]:
        return '*'
    else: 
        return ''

def diff_means(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

def compare_facets(df, facets, cols, statistic=diff_means):
    '''Compare statistic in cols of df via permutation. Display results, return nothing.'''
    for facet in facets:
        print(f"\n==============\nFacet: {facet}\n==============")
        df = df.dropna(subset=[facet])
        for level in df[facet].unique():
            data = (df.loc[df[facet]==level, cols], df.loc[~(df[facet]==level), cols])
            level_mean = np.mean(data[0], axis=0)
            other_mean = np.mean(data[1], axis=0)
            output = pd.DataFrame(level_mean, columns=[level]).join(pd.DataFrame(other_mean, columns=['others']))
            res = permutation_test(data, statistic)
            output['diff'] = res.statistic
            output['p'] = res.pvalue
            output['sig'] = output['p'].apply(significance_label)
            display(output)

In [7]:
compare_facets(df, facet_by, cols_to_test, diff_means)


Facet: inf_gender


,she/her,others,diff,p,sig
char_count,3752.279775,3844.495428,-92.215652,0.3942,
num_gpe_places,16.356180,21.814954,-5.458775,0.0002,***
num_nongpe_places,122.757303,123.285637,-0.528334,0.8536,
num_all_places,138.620225,144.524476,-5.904251,0.1032,
dist_miles,56148.685011,79583.242665,-23434.557654,0.0002,***
Tokens,119516.989888,131900.656267,-12383.666379,0.0002,***
num_gpe_places_norm,0.000141,0.000170,-0.000029,0.0002,***
num_nongpe_places_norm,0.001127,0.001048,0.000079,0.0022,**
num_gpe_places_norm_byCharacter,0.005224,0.009569,-0.004345,0.0002,***
num_nongpe_places_norm_byCharacter,0.036178,0.037657,-0.001479,0.0378,*


,he/him/his,others,diff,p,sig
char_count,4012.726801,3475.705128,537.021673,0.0002,***
num_gpe_places,22.025360,16.663708,5.361652,0.0002,***
num_nongpe_places,127.342939,115.879684,11.463255,0.0002,***
num_all_places,148.785591,132.051282,16.734309,0.0002,***
dist_miles,80558.158976,57346.323422,23211.835554,0.0002,***
Tokens,133472.337752,118342.144970,15130.192782,0.0002,***
num_gpe_places_norm,0.000169,0.000146,0.000023,0.0002,***
num_nongpe_places_norm,0.001073,0.001074,-0.000001,0.9646,
num_gpe_places_norm_byCharacter,0.008396,0.007762,0.000634,0.1698,
num_nongpe_places_norm_byCharacter,0.036880,0.037687,-0.000807,0.2578,


,they/them/their,others,diff,p,sig
char_count,1490.612903,3924.422857,-2433.809954,0.0002,***
num_gpe_places,18.870968,20.103238,-1.232270,0.5638,
num_nongpe_places,66.516129,125.788190,-59.272061,0.0002,***
num_all_places,84.903226,145.339048,-60.435822,0.0002,***
dist_miles,65942.276537,72282.184946,-6339.908409,0.6012,
Tokens,109909.790323,128740.810286,-18831.019963,0.0006,***
num_gpe_places_norm,0.000183,0.000160,0.000024,0.0814,
num_nongpe_places_norm,0.000695,0.001091,-0.000396,0.0002,***
num_gpe_places_norm_byCharacter,0.025981,0.007321,0.018660,0.0002,***
num_nongpe_places_norm_byCharacter,0.048523,0.036642,0.011881,0.0002,***



Facet: Category


...

,PW,others,diff,p,sig
char_count,3177.618677,3880.336276,-702.717599,0.0002,***
num_gpe_places,14.801556,20.588684,-5.787127,0.0002,***
num_nongpe_places,121.428016,123.288523,-1.860508,0.7184,
num_all_places,135.782101,143.317416,-7.535315,0.1962,
dist_miles,52226.899827,74035.017864,-21808.118037,0.0010,**
Tokens,122261.315175,128472.023676,-6210.708501,0.1568,
num_gpe_places_norm,0.000134,0.000163,-0.000029,0.0010,**
num_nongpe_places_norm,0.001108,0.001070,0.000038,0.3292,
num_gpe_places_norm_byCharacter,0.005194,0.008469,-0.003275,0.0002,***
num_nongpe_places_norm_byCharacter,0.042319,0.036648,0.005672,0.0002,***


,MIX,others,diff,p,sig
char_count,1869.167539,3959.904222,-2090.736683,0.0002,***
num_gpe_places,25.450262,19.644253,5.806008,0.0010,**
num_nongpe_places,84.507853,125.997263,-41.489410,0.0002,***
num_all_places,109.403141,145.092651,-35.689509,0.0002,***
dist_miles,103312.422699,69657.898764,33654.523935,0.0008,***
Tokens,126088.335079,128026.023847,-1937.688768,0.7154,
num_gpe_places_norm,0.000204,0.000157,0.000047,0.0002,***
num_nongpe_places_norm,0.000745,0.001098,-0.000352,0.0002,***
num_gpe_places_norm_byCharacter,0.018654,0.007379,0.011275,0.0002,***
num_nongpe_places_norm_byCharacter,0.046469,0.036484,0.009985,0.0002,***


,MEM,others,diff,p,sig
char_count,4260.253275,3774.146032,486.107243,0.0112,*
num_gpe_places,37.877729,18.427381,19.450348,0.0002,***
num_nongpe_places,170.908297,118.771429,52.136868,0.0002,***
num_all_places,207.737991,136.694841,71.043150,0.0002,***
dist_miles,159239.594297,64068.139159,95171.455138,0.0002,***
Tokens,105611.161572,129916.065476,-24304.903904,0.0002,***
num_gpe_places_norm,0.000355,0.000143,0.000212,0.0002,***
num_nongpe_places_norm,0.001666,0.001019,0.000646,0.0002,***
num_gpe_places_norm_byCharacter,0.009739,0.008019,0.001720,0.0466,*
num_nongpe_places_norm_byCharacter,0.041595,0.036777,0.004819,0.0004,***


,MID,others,diff,p,sig
char_count,3103.355422,3860.351916,-756.996495,0.0002,***
num_gpe_places,7.024096,20.884630,-13.860534,0.0002,***
num_nongpe_places,94.024096,124.984127,-30.960031,0.0002,***
num_all_places,100.753012,145.303136,-44.550124,0.0002,***
dist_miles,16320.877678,75574.259419,-59253.381741,0.0002,***
Tokens,84156.487952,130702.076655,-46545.588703,0.0002,***
num_gpe_places_norm,0.000089,0.000165,-0.000076,0.0002,***
num_nongpe_places_norm,0.001243,0.001062,0.000181,0.0002,***
num_gpe_places_norm_byCharacter,0.002412,0.008532,-0.006120,0.0002,***
num_nongpe_places_norm_byCharacter,0.031853,0.037520,-0.005667,0.0002,***


,BIO,others,diff,p,sig
char_count,4291.549223,3778.629499,512.919724,0.0160,*
num_gpe_places,59.735751,17.050861,42.684891,0.0002,***
num_nongpe_places,175.652850,119.147496,56.505354,0.0002,***
num_all_places,233.875648,135.721831,98.153817,0.0002,***
dist_miles,252167.099482,58391.755702,193775.343780,0.0002,***
Tokens,196532.989637,122708.362285,73824.627352,0.0002,***
num_gpe_places_norm,0.000331,0.000148,0.000184,0.0002,***
num_nongpe_places_norm,0.001036,0.001076,-0.000040,0.3888,
num_gpe_places_norm_byCharacter,0.015934,0.007576,0.008359,0.0002,***
num_nongpe_places_norm_byCharacter,0.046214,0.036496,0.009718,0.0002,***


## GPE distances

### Geonames

In [8]:
gpe_cols = ['gpe_places', 'num_gpe_places', 'gpe_sequences', 'dist_miles']
df.loc['Woman Behind the New Deal, The - Kirstin Downey', 'gpe_places']

"['Columbia', 'Cornell', 'Cornell', 'New York City', 'Maine', 'Boston Common', 'Newcastle', 'Worcester', 'Worcester', 'Massachusetts', 'Virginia', 'New York City', 'New York City', 'Worcester', 'Worcester', 'Illinois', 'Chicago', 'Lake Forest', 'Lake Forest', 'Lake Forest', 'Lake Forest', 'Chicago', 'Chicago', 'Chicago', 'Chicago', 'Chicago', 'Chicago', 'Chicago', 'Chicago', 'Chicago', 'Chicago', 'Penn', 'Penn', 'Penn', 'Columbia', 'New York', 'New Jersey', 'New York City', 'Mount Holyoke', 'Philadelphia', 'Philadelphia', 'Philadelphia', 'Philadelphia', 'Philadelphia', 'New York City', 'New York City', 'New York City', 'New York Times', 'New York', 'Albany', 'Albany', 'Albany', 'New York', 'New York', 'New York', 'New York', 'New York', 'New York', 'Albany', 'Albany', 'Albany', 'Albany', 'Albany', 'Albany', 'Albany', 'Albany', 'New York City', 'New York City', 'Lake Forest', 'Lake Forest', 'Chicago', 'Chicago', 'Chicago', 'New York City', 'New York City', 'New York City', 'Long Island 

In [9]:
df.loc['Woman Behind the New Deal, The - Kirstin Downey', 'gpe_sequences']

"['Columbia', 'Cornell', 'New York City', 'Maine', 'Boston Common', 'Newcastle', 'Worcester', 'Massachusetts', 'Virginia', 'New York City', 'Worcester', 'Illinois', 'Chicago', 'Lake Forest', 'Chicago', 'Penn', 'Columbia', 'New York', 'New Jersey', 'New York City', 'Mount Holyoke', 'Philadelphia', 'New York City', 'New York Times', 'New York', 'Albany', 'New York', 'Albany', 'New York City', 'Lake Forest', 'Chicago', 'New York City', 'Long Island Sound', 'New York', 'the United States', 'New York', 'Chicago', 'New York', 'in Garden City , New York', 'New York', 'Washington', 'New York', 'New Rochelle', 'New York', 'Rome', 'Scranton', 'New York City', 'Albany', 'Manhattan', 'New York City', 'Albany', 'San Francisco', 'Rochester', 'Houston', 'New York City', 'Albany', 'New York', 'Cattaraugus County', 'New York', 'Albany', 'New York', 'Albany', 'New York City', 'New York', 'Warm Springs', 'New York', 'Washington', 'New York', 'New York City', 'New York', 'New York City', 'New York', 'Wash

It **looks** like `gpe_places` is the observed sequence of GPEs in the source text, while `gpe_sequences` de-dupes (naïvely) immediate repetitions (but not repetitions with an intervening place). **We should confirm this with Sandeep if we're not sure.** 

In [10]:
gn_col_names = [
    'geonameid',
    'name',
    'asciiname',
    'alternatenames',
    'lat',
    'lon',
    'feature_class',
    'feature_code',
    'country_code',
    'cc2',
    'admin1_code',
    'admin2_code',
    'admin3_code',
    'admin4_code',
    'population',
    'elevation',
    'dem',
    'timezone',
    'mod_date'
]
gn = pd.read_csv(
    os.path.join(data_dir, 'geonames', 'allCountries.zip'), 
    sep='\t',
    names=gn_col_names,
    index_col=0
)
gn.sample(5)

/var/folders/7w/9w42413d6yq9tdnnb2kd8wgc0000gn/T/ipykernel_11286/2729711462.py:22: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  gn = pd.read_csv(


,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
10914580,Fanrongcun,Fanrongcun,"Fanrongcun,fan rong cun,繁荣村",43.82393,125.14517,P,PPL,CN,NaN,5.0,2201.0,NaN,NaN,0,NaN,199,Asia/Shanghai,2023-08-15
1607851,Nong Phak Nam Lang,Nong Phak Nam Lang,NaN,12.93333,101.33333,H,MRSH,TH,NaN,47.0,NaN,NaN,NaN,0,NaN,45,Asia/Bangkok,1994-04-06
2151810,Reedy Creek,Reedy Creek,NaN,-35.80000,150.23333,H,STM,AU,NaN,2,NaN,NaN,NaN,0,NaN,1,Australia/Sydney,1993-12-30
8301608,Thalgahagama,Thalgahagama,NaN,7.54890,80.21990,A,ADM4,LK,NaN,32.0,61.0,6149.0,6149055.0,0,NaN,75,Asia/Colombo,2018-12-13
7515770,Dacha,Dacha,"Dacha,da cha,大岔",36.00590,106.45445,P,PPL,CN,NaN,21.0,6404.0,NaN,NaN,0,NaN,1691,Asia/Shanghai,2021-09-20


In [11]:
hierarchy = pd.read_csv(
    os.path.join(data_dir, 'geonames', 'hierarchy.zip'), 
    sep='\t',
    names=['parent', 'child', 'type']
)
hierarchy

,parent,child,type
0,6295630,6255146,ADM
1,6295630,6255152,ADM
2,6295630,6255147,ADM
3,6295630,6255148,ADM
4,6295630,6255149,ADM
...,...,...,...
492650,648605,12557021,NaN
492651,12558102,3923974,NaN
492652,1680578,1712681,NaN
492653,3996063,8581707,NaN


In [18]:
feature_codes = pd.read_csv(
    os.path.join(data_dir, 'geonames', 'featureCodes_en.txt'),
    sep='\t',
    names=['feature_code', 'feature_name', 'feature_description']
)
feature_codes

,feature_code,feature_name,feature_description
0,A.ADM1,first-order administrative division,a primary administrative division of a country...
1,A.ADM1H,historical first-order administrative division,a former first-order administrative division
2,A.ADM2,second-order administrative division,a subdivision of a first-order administrative ...
3,A.ADM2H,historical second-order administrative division,a former second-order administrative division
4,A.ADM3,third-order administrative division,a subdivision of a second-order administrative...
...,...,...,...
676,V.TREE,tree(s),a conspicuous tree used as a landmark
677,V.TUND,tundra,"a marshy, treeless, high latitude plain, domin..."
678,V.VIN,vineyard,a planting of grapevines
679,V.VINS,vineyards,plantings of grapevines


In [12]:
gn.loc[gn.feature_code=='ADM1'].sample(5)

,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
4566106,Luquillo Municipio,Luquillo Municipio,"LUQ,Luquillo",18.33967,-65.72461,A,ADM1,PR,NaN,89.0,NaN,NaN,NaN,20068,125.0,111,America/Puerto_Rico,2019-04-14
3983035,Estado de Sinaloa,Estado de Sinaloa,"Estado de Sinaloa,SIN,Sinaloa,shinaroa zhou,xi...",25.00000,-107.50000,A,ADM1,MX,NaN,25,NaN,NaN,NaN,3026943,NaN,139,America/Mazatlan,2022-12-05
142550,West Azerbaijan Province,West Azerbaijan Province,"Azarbaijan Gharbi,Azarbayjan-e Baxtari,Azarbay...",37.66667,45.00000,A,ADM1,IR,NaN,1.0,NaN,NaN,NaN,3265219,NaN,1350,Asia/Tehran,2021-01-13
1855608,Nara-ken,Nara-ken,"Nara,Nara Prefecture,nai liang,nai liang xian,...",34.68525,135.83289,A,ADM1,JP,NaN,28,NaN,NaN,NaN,1321805,NaN,96,Asia/Tokyo,2023-07-18
460311,Dobeles novads,Dobeles novads,"Dobel'skiy Rayon,Dobele Municipality,Dobeles R...",56.58333,23.16667,A,ADM1,LV,NaN,08,NaN,NaN,NaN,28517,NaN,88,Europe/Riga,2023-07-08


In [25]:
gn.loc[(gn.name=='Paris') & (gn.population>0)].sort_values(by='population', ascending=False)

,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
2988506,Paris,Paris,"Paras,Parigi,Paris,París,Páras",48.85339,2.34864,A,ADM3,FR,NaN,11,75,751,NaN,4380654,NaN,35,Europe/Paris,2022-03-27
6455259,Paris,Paris,"75056,Paris",48.86000,2.34444,A,ADM4,FR,NaN,11,75,751,75056,2190327,NaN,54,Europe/Paris,2019-03-23
2968815,Paris,Paris,"Departement de Paris,Département de Paris,Pari...",48.85340,2.34860,A,ADM2,FR,NaN,11,75,NaN,NaN,2165423,NaN,35,Europe/Paris,2022-03-10
2988507,Paris,Paris,"Baariis,Bahliz,Baris,Lungsod ng Paris,Lutece,L...",48.85341,2.34880,P,PPLC,FR,NaN,11,75,751,75056,2138551,NaN,42,Europe/Paris,2023-09-22
4717560,Paris,Paris,"PRX,Paris,Parizh,barys,parys tgzas,Париж,Пари...",33.66094,-95.55551,P,PPLA2,US,NaN,TX,277,NaN,NaN,24782,183.0,190,America/Chicago,2017-03-09
6942553,Paris,Paris,پیرس، انٹاریو,43.20000,-80.38333,P,PPL,CA,NaN,8.0,NaN,NaN,NaN,12310,NaN,255,America/Toronto,2021-12-21
4647963,Paris,Paris,"PHT,Paris,Parizh,barys,parys tnsy,Париж,Парис...",36.30200,-88.32671,P,PPLA2,US,NaN,TN,79,NaN,NaN,10150,157.0,158,America/Chicago,2017-03-09
4303602,Paris,Paris,"Paris,Parizh,ba li,barys,parys kntaky,Париж,П...",38.20980,-84.25299,P,PPLA2,US,NaN,KY,17,NaN,NaN,9870,257.0,258,America/New_York,2017-03-09
4246659,Paris,Paris,"Parizh,Peris,barys,parys aylynwy,Париж,Перис,...",39.61115,-87.69614,P,PPLA2,US,NaN,IL,45,57641.0,NaN,8432,220.0,223,America/Chicago,2017-05-23


### Hop mobility

In [66]:
hop = pd.read_json(os.path.join(data_dir, 'derived', 'mb.hoplist.gpe.all.jsonl.bz2'), lines=True)
hop

,book_id,chars
0,00004005,"[{'char_id': 163, 'sequence': [{'place': 'Long..."
1,00020690,"[{'char_id': 0, 'sequence': [{'place': 'Detroi..."
2,00023581,"[{'char_id': 189, 'sequence': [{'place': 'Oldo..."
3,eaf172,"[{'char_id': 0, 'sequence': [{'place': 'Boston..."
4,00024963,"[{'char_id': 340, 'sequence': [{'place': 'Virg..."
...,...,...
10688,00023240,"[{'char_id': 42, 'sequence': [{'place': 'Wisco..."
10689,00020856,"[{'char_id': 234, 'sequence': [{'place': 'Ambr..."
10690,00024395,"[{'char_id': 359, 'sequence': [{'place': 'Gene..."
10691,00023467,"[{'char_id': 0, 'sequence': [{'place': 'London..."


### Get sequences of places and sets of unique places

In [26]:
df.gpe_sequences

book_id
[Heist Society 2] Uncommon Criminals - Ally Carter    ['Paraguay', 'London', 'New York City', 'midto...
2001_2011_Wilson,RobertCharles_TheChronoliths_SF      ['Bangkok', 'Minneapolis / St. Paul', 'Chumpho...
2001_Martel,Yann_LifeofPi_BS                          ['Canada', 'India', 'Canada', 'India', 'Toront...
2002_2011_Anderson,MT_Feed_SF                         ['Lonely', 'School', 'Switzerland', 'Io', 'Ame...
2002_Baker,Jo_Offcomer_CT                             ['Belfast', 'Conroys', 'October', 'Belfast', '...
                                                                            ...                        
Woman Behind the New Deal, The - Kirstin Downey       ['Columbia', 'Cornell', 'New York City', 'Main...
Woman of No Importance, A - Sonia Purnell             ['France', 'Europe', 'France', 'Europe', 'Camb...
Woodrow Wilson - John Milton Cooper, Jr_              ['America', 'Washington', 'Tennessee', 'German...
You Never Forget Your First - Alexis Coe              ['

In [54]:
gpe_occurs = []
for i in df.gpe_places:
    if i != "['nan']":
        #print(i)
        gpe_occurs += literal_eval(i)
gpe_unique = set(gpe_occurs)

['Paraguay', 'Paraguay', 'London', 'New York City', 'midtown Manhattan', 'London', 'London', 'Hale', 'Lyon', 'Loxley', 'Monaco', 'Monaco', 'Belize', 'Belize', 'Edinburgh', 'Monaco', 'Monaco', 'Monaco', 'Monte - Carlo', 'Brazil', 'Monte Carlo', 'Monte Carlo', 'Austria', 'Austria', 'Austria', 'Lyon', 'Lyon', 'Lyon', 'Lyon', 'Lyon', 'New York', 'Monaco']
['Bangkok', 'Minneapolis / St. Paul', 'Chumphon', 'Chumphon', 'Bangkok', 'Bangkok', 'Cornell', 'Campion - Miller', 'Campion - Miller', 'Campion - Miller', 'Edina', 'Campion - Miller', 'Campion - Miller', 'Minneapolis', 'Minneapolis', 'Chumphon', 'Chumphon', 'Maryland', 'Maryland', 'Nashua', 'New Hampshire', 'New York City', 'Israel', 'Israel', 'Baltimore', 'Cornell', 'Chumphon', 'Jerusalem', 'Baltimore', 'Baltimore', 'Baltimore', 'Minnesota', 'Baltimore', 'Chumphon', 'Chumphon', 'Thailand', 'Minneapolis', 'Minneapolis', 'Portillo', 'Cairo', 'Tucson', 'Tucson', 'Jerusalem', 'Jerusalem', 'Portillo', 'Portillo', 'Portillo', 'Portillo', 'Port

ValueError: malformed node or string on line 1: <ast.Name object at 0x14d3072b0>

In [48]:
gpe_occurs[:100]

['Paraguay',
 'Paraguay',
 'London',
 'New York City',
 'midtown Manhattan',
 'London',
 'London',
 'Hale',
 'Lyon',
 'Loxley',
 'Monaco',
 'Monaco',
 'Belize',
 'Belize',
 'Edinburgh',
 'Monaco',
 'Monaco',
 'Monaco',
 'Monte - Carlo',
 'Brazil',
 'Monte Carlo',
 'Monte Carlo',
 'Austria',
 'Austria',
 'Austria',
 'Lyon',
 'Lyon',
 'Lyon',
 'Lyon',
 'Lyon',
 'New York',
 'Monaco',
 'Bangkok',
 'Minneapolis / St. Paul',
 'Chumphon',
 'Chumphon',
 'Bangkok',
 'Bangkok',
 'Cornell',
 'Campion - Miller',
 'Campion - Miller',
 'Campion - Miller',
 'Edina',
 'Campion - Miller',
 'Campion - Miller',
 'Minneapolis',
 'Minneapolis',
 'Chumphon',
 'Chumphon',
 'Maryland',
 'Maryland',
 'Nashua',
 'New Hampshire',
 'New York City',
 'Israel',
 'Israel',
 'Baltimore',
 'Cornell',
 'Chumphon',
 'Jerusalem',
 'Baltimore',
 'Baltimore',
 'Baltimore',
 'Minnesota',
 'Baltimore',
 'Chumphon',
 'Chumphon',
 'Thailand',
 'Minneapolis',
 'Minneapolis',
 'Portillo',
 'Cairo',
 'Tucson',
 'Tucson',
 'Jerus